In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import os
import re
import numpy as np
import pandas as pd
import collections

In [2]:
x_name = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터3] samp_cst_feat'}.csv")
df_x = pd.read_csv(x_name, index_col=0)

y_name = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터2] samp_train'}.csv")
df_y = pd.read_csv(y_name, index_col=0)

df = pd.merge(df_x, df_y, on='cst_id_di')

In [3]:
df.MRC_ID_DI[df.MRC_ID_DI > 0 ] = 1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [4]:
nc_name = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터4] variable_dtype'}.xlsx")
nc = pd.read_excel(nc_name, index_col=0)

In [ ]:
df

In [ ]:
%matplotlib inline
plt.figure(figsize = (227, 227))
sns.heatmap(df.corr(), annot = True, fmt = '.2f', square = True)
plt.savefig('corr13.png')
plt.show()

In [ ]:
pd.set_option('display.max_columns', None)
df.describe([.01, .10, .20, .30, .40, .50, .60, .70, .80, .90, .95, .99])

In [ ]:
for i in df.columns[0:]:
    if i == 'MRC_ID_DI' or nc.loc[i, 'dType'] == 'categorical':
        ax = sns.countplot(df[i], palette=['#432371',"#FAAE7B"])
        plt.show()
    else:
        ax = sns.boxplot(x=i, data=df, color = 'yellow')
        ax.set(xlim=(-1, 1))
        plt.show()

In [ ]:
#https://www.pluralsight.com/guides/cleaning-up-data-from-outliers
#https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba
Q1 = df.quantile(0.05)
Q3 = df.quantile(0.95)
IQR = Q3 - Q1
df_out = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]
print(df_out.shape)

In [ ]:
#https://www.kite.com/python/answers/how-to-remove-outliers-from-a-pandas-dataframe-in-python
from scipy import stats
import numpy as np

z_scores = stats.zscore(df)

abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 5).all(axis=1)
df = df[filtered_entries]
print(df.shape)

In [5]:
"""
drop_columns = list()
for i in df.columns[0:-1]:
    if nc.loc[i, 'dType'] == 'numerical':
        if df[i].quantile(0.75) - df[i].quantile(0.25) > 0.5:
            print(i)
            drop_columns.append(i)
    else:
        if len(df[df[i] == 1]) > len(df)*0.8 or len(df[df[i] == 0]) > len(df)*0.8:
            drop_columns.append(i)

df = df.drop(columns = drop_columns, axis=1)
"""
#https://www.pluralsight.com/guides/cleaning-up-data-from-outliers
for i in df.columns[0:-1]:
    if nc.loc[i, 'dType'] == 'numerical':
        d_90 = df[i].quantile(0.99)
        d_10 = df[i].quantile(0.01)
        d_50 = df[i].quantile(0.50)
        df[i] = np.where(df[i] > d_90, d_90, df[i])
        df[i] = np.where(df[i] < d_10, d_10, df[i])
print(df.shape)

(10124, 227)


In [6]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import random

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

seed_value= 0
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [7]:
import sys
ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                      exception_only=False, running_compiled_code=False):
       etype, value, tb = sys.exc_info()
       return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

In [153]:
df_0 = df[df['MRC_ID_DI'] == 0].sample(frac=1)
df_1 = df[df['MRC_ID_DI'] == 1].sample(frac=1)

sample_size = len(df_0) if len(df_0) < len(df_1) else len(df_1)

df_h = pd.concat([df_0.head(sample_size), df_1.head(sample_size)]).sample(frac=1)
df_t = pd.concat([df_0.tail(sample_size), df_1.head(sample_size)]).sample(frac=1)
df_f = pd.concat([df_h, df_t]).sample(frac=1)

In [154]:
X = df_f.drop(columns = ['MRC_ID_DI', 'VAR021', 'VAR046'], axis=1)
y = tf.keras.utils.to_categorical(df_f['MRC_ID_DI'])

In [155]:
num, cat = list(), list()
for i in X.columns:
    if nc.loc[i, 'dType'] == 'numerical':
        num.append(i)
    else:
        if len(df[df[i] == 1]) > len(df)*0.8 or len(df[df[i] == 0]) > len(df)*0.8:
            continue
        else:
            #cat.append(i)
            num.append(i)
X = X[num]

for i in X.columns:
    if nc.loc[i, 'dType'] == 'numerical':
        #if 8*X[i].quantile(1) < 1 and  8*X[i].quantile(0) > -1:
        #    X[i] *= 8
        #elif 4*X[i].quantile(1) < 1 and  4*X[i].quantile(0) > -1:
        #    X[i] *= 4
        #elif 3*X[i].quantile(1) < 1 and  3*X[i].quantile(0) > -1:
        #    X[i] *= 3
        if 2*X[i].quantile(1) < 1 and  2*X[i].quantile(0) > -1:
            X[i] *= 2
        
#X[X.select_dtypes(include=['number']).columns] *= 10

In [156]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [157]:
"""
X1_train = X_train[num]
X1_test = X_test[num]

X2_train = X_train[cat]
X2_test = X_test[cat]
"""

'\nX1_train = X_train[num]\nX1_test = X_test[num]\n\nX2_train = X_train[cat]\nX2_test = X_test[cat]\n'

In [158]:
"""
input_1 = tf.keras.Input(dtype = tf.float32, shape = (len(num),))
input_2 = tf.keras.Input(dtype = tf.float32, shape = (len(cat),))

dense_layer_1_1 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(input_1)
dense_layer_1_2 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_1_1)
dense_layer_1_3 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_1_2)
dense_layer_1_4 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_1_3)
dropout_1_5 = tf.keras.layers.Dropout(rate = 0.2)(dense_layer_1_2)


dense_layer_2_1 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(input_2)
dense_layer_2_2 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_2_1)
dense_layer_2_3 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_2_2)
dense_layer_2_4 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_2_3)
dropout_2_5 = tf.keras.layers.Dropout(rate = 0.2)(dense_layer_2_4)

concat_layer = tf.keras.layers.Concatenate()([dropout_1_5, dropout_2_5])


dense_layer_3 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(concat_layer)
dense_layer_4 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_3)
dense_layer_5 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_4)
dense_layer_6 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_5)

output = tf.keras.layers.Dense(units = 2, activation = tf.nn.softmax)(dense_layer_4)
"""

'\ninput_1 = tf.keras.Input(dtype = tf.float32, shape = (len(num),))\ninput_2 = tf.keras.Input(dtype = tf.float32, shape = (len(cat),))\n\ndense_layer_1_1 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(input_1)\ndense_layer_1_2 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_1_1)\ndense_layer_1_3 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_1_2)\ndense_layer_1_4 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_1_3)\ndropout_1_5 = tf.keras.layers.Dropout(rate = 0.2)(dense_layer_1_2)\n\n\ndense_layer_2_1 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(input_2)\ndense_layer_2_2 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_2_1)\ndense_layer_2_3 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_2_2)\ndense_layer_2_4 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_2_3)\ndropout_2_5 = tf.keras.layers.Dropout(rate = 

In [159]:
input_ = tf.keras.Input(dtype = tf.float32, shape = (len(X.columns),))

dense_layer_1_1 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(input_)
dense_layer_1_2 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_1_1)
dense_layer_1_3 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_1_2)
dense_layer_1_4 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(dense_layer_1_3)

dropout_1_5 = tf.keras.layers.Dropout(rate = 0.2)(dense_layer_1_1)

output = tf.keras.layers.Dense(units = 2, activation = tf.nn.softmax)(dropout_1_5)

In [160]:
model = tf.keras.Model(inputs=input_, outputs=output)
#model = tf.keras.Model(inputs=[input_1, input_2], outputs=output)

In [161]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=0.00005), metrics=['acc'])
print(model.summary())

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 204)]             0         
_________________________________________________________________
dense_40 (Dense)             (None, 10)                2050      
_________________________________________________________________
dropout_8 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_44 (Dense)             (None, 2)                 22        
Total params: 2,072
Trainable params: 2,072
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#history = model.fit(x=X_train, y=y_train, batch_size=32, epochs=400, verbose=1, validation_split=0.1)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=100)
history = model.fit(x=X_train, y=y_train, batch_size=32, epochs=10000, verbose=1, validation_split=0.1, callbacks=[callback])
#history = model.fit(x=[X1_train, X2_train], y=y_train, batch_size=32, epochs=2000, verbose=1, validation_split=0.1, callbacks=[callback])

Train on 6233 samples, validate on 693 samples
Epoch 1/10000
6233/6233 [==============================] - 1s 87us/sample - loss: 0.6474 - acc: 0.6288 - val_loss: 0.6230 - val_acc: 0.6681
Epoch 2/10000
6233/6233 [==============================] - 0s 51us/sample - loss: 0.6050 - acc: 0.6745 - val_loss: 0.5913 - val_acc: 0.6898
Epoch 3/10000
6233/6233 [==============================] - 0s 55us/sample - loss: 0.5769 - acc: 0.6993 - val_loss: 0.5705 - val_acc: 0.7071
Epoch 4/10000
6233/6233 [==============================] - 0s 48us/sample - loss: 0.5563 - acc: 0.7226 - val_loss: 0.5545 - val_acc: 0.7128
Epoch 5/10000
6233/6233 [==============================] - 0s 48us/sample - loss: 0.5459 - acc: 0.7271 - val_loss: 0.5425 - val_acc: 0.7258
Epoch 6/10000
6233/6233 [==============================] - 0s 49us/sample - loss: 0.5307 - acc: 0.7393 - val_loss: 0.5327 - val_acc: 0.7330
Epoch 7/10000
6233/6233 [==============================] - 0s 64us/sample - loss: 0.5279 - acc: 0.7404 - val_loss

Epoch 59/10000
6233/6233 [==============================] - 0s 54us/sample - loss: 0.4431 - acc: 0.7982 - val_loss: 0.4319 - val_acc: 0.7893
Epoch 60/10000
6233/6233 [==============================] - 0s 53us/sample - loss: 0.4423 - acc: 0.7998 - val_loss: 0.4311 - val_acc: 0.7864
Epoch 61/10000
6233/6233 [==============================] - 0s 52us/sample - loss: 0.4424 - acc: 0.7970 - val_loss: 0.4303 - val_acc: 0.7879
Epoch 62/10000
6233/6233 [==============================] - 0s 49us/sample - loss: 0.4433 - acc: 0.7970 - val_loss: 0.4298 - val_acc: 0.7908
Epoch 63/10000
6233/6233 [==============================] - 0s 52us/sample - loss: 0.4391 - acc: 0.8001 - val_loss: 0.4294 - val_acc: 0.7922
Epoch 64/10000
6233/6233 [==============================] - 0s 53us/sample - loss: 0.4394 - acc: 0.7966 - val_loss: 0.4294 - val_acc: 0.7893
Epoch 65/10000
6233/6233 [==============================] - 0s 53us/sample - loss: 0.4390 - acc: 0.7996 - val_loss: 0.4287 - val_acc: 0.7908
Epoch 66/1000

Epoch 117/10000
6233/6233 [==============================] - 0s 50us/sample - loss: 0.4277 - acc: 0.8046 - val_loss: 0.4137 - val_acc: 0.7937
Epoch 118/10000
6233/6233 [==============================] - 0s 49us/sample - loss: 0.4213 - acc: 0.8108 - val_loss: 0.4131 - val_acc: 0.7951
Epoch 119/10000
6233/6233 [==============================] - 0s 49us/sample - loss: 0.4153 - acc: 0.8099 - val_loss: 0.4131 - val_acc: 0.7951
Epoch 120/10000
6233/6233 [==============================] - 0s 52us/sample - loss: 0.4227 - acc: 0.8124 - val_loss: 0.4128 - val_acc: 0.7937
Epoch 121/10000
6233/6233 [==============================] - 0s 50us/sample - loss: 0.4211 - acc: 0.8123 - val_loss: 0.4121 - val_acc: 0.7937
Epoch 122/10000
6233/6233 [==============================] - 0s 50us/sample - loss: 0.4212 - acc: 0.8110 - val_loss: 0.4119 - val_acc: 0.7965
Epoch 123/10000
6233/6233 [==============================] - 0s 50us/sample - loss: 0.4197 - acc: 0.8102 - val_loss: 0.4119 - val_acc: 0.7937
Epoch 

Epoch 175/10000
6233/6233 [==============================] - 0s 56us/sample - loss: 0.4122 - acc: 0.8149 - val_loss: 0.4036 - val_acc: 0.7937
Epoch 176/10000
6233/6233 [==============================] - 0s 50us/sample - loss: 0.4105 - acc: 0.8190 - val_loss: 0.4036 - val_acc: 0.7951
Epoch 177/10000
6233/6233 [==============================] - 0s 51us/sample - loss: 0.4087 - acc: 0.8133 - val_loss: 0.4031 - val_acc: 0.7922
Epoch 178/10000
6233/6233 [==============================] - 0s 50us/sample - loss: 0.4094 - acc: 0.8161 - val_loss: 0.4032 - val_acc: 0.7937
Epoch 179/10000
6233/6233 [==============================] - 0s 57us/sample - loss: 0.4103 - acc: 0.8145 - val_loss: 0.4029 - val_acc: 0.7951
Epoch 180/10000
6233/6233 [==============================] - 0s 55us/sample - loss: 0.4112 - acc: 0.8145 - val_loss: 0.4029 - val_acc: 0.7965
Epoch 181/10000
6233/6233 [==============================] - 0s 53us/sample - loss: 0.4122 - acc: 0.8149 - val_loss: 0.4033 - val_acc: 0.7951
Epoch 

Epoch 233/10000
6233/6233 [==============================] - 0s 54us/sample - loss: 0.3994 - acc: 0.8192 - val_loss: 0.3984 - val_acc: 0.7937
Epoch 234/10000
6233/6233 [==============================] - 0s 53us/sample - loss: 0.3999 - acc: 0.8184 - val_loss: 0.3980 - val_acc: 0.7922
Epoch 235/10000
6233/6233 [==============================] - 0s 50us/sample - loss: 0.4053 - acc: 0.8179 - val_loss: 0.3975 - val_acc: 0.7937
Epoch 236/10000
6233/6233 [==============================] - 0s 50us/sample - loss: 0.4028 - acc: 0.8214 - val_loss: 0.3980 - val_acc: 0.7965
Epoch 237/10000
6233/6233 [==============================] - 0s 50us/sample - loss: 0.4019 - acc: 0.8150 - val_loss: 0.3974 - val_acc: 0.7893
Epoch 238/10000
6233/6233 [==============================] - 0s 50us/sample - loss: 0.4033 - acc: 0.8163 - val_loss: 0.3971 - val_acc: 0.7951
Epoch 239/10000
6233/6233 [==============================] - 0s 51us/sample - loss: 0.4054 - acc: 0.8182 - val_loss: 0.3974 - val_acc: 0.7937
Epoch 

...

6233/6233 [==============================] - 0s 55us/sample - loss: 0.3903 - acc: 0.8259 - val_loss: 0.3914 - val_acc: 0.7922
Epoch 349/10000
6233/6233 [==============================] - 0s 50us/sample - loss: 0.3957 - acc: 0.8206 - val_loss: 0.3907 - val_acc: 0.7965
Epoch 350/10000
6233/6233 [==============================] - 0s 50us/sample - loss: 0.3903 - acc: 0.8287 - val_loss: 0.3914 - val_acc: 0.7922
Epoch 351/10000
6233/6233 [==============================] - 0s 53us/sample - loss: 0.3903 - acc: 0.8274 - val_loss: 0.3909 - val_acc: 0.7908
Epoch 352/10000
6233/6233 [==============================] - 0s 51us/sample - loss: 0.3902 - acc: 0.8259 - val_loss: 0.3911 - val_acc: 0.7951
Epoch 353/10000
6233/6233 [==============================] - 0s 51us/sample - loss: 0.3908 - acc: 0.8303 - val_loss: 0.3906 - val_acc: 0.7980
Epoch 354/10000
6233/6233 [==============================] - 0s 60us/sample - loss: 0.3928 - acc: 0.8261 - val_loss: 0.3908 - val_acc: 0.7965
Epoch 355/10000
6233/6

Epoch 406/10000
6233/6233 [==============================] - 0s 50us/sample - loss: 0.3822 - acc: 0.8303 - val_loss: 0.3885 - val_acc: 0.7994
Epoch 407/10000
6233/6233 [==============================] - 0s 50us/sample - loss: 0.3825 - acc: 0.8270 - val_loss: 0.3882 - val_acc: 0.8009
Epoch 408/10000
6233/6233 [==============================] - 0s 49us/sample - loss: 0.3873 - acc: 0.8290 - val_loss: 0.3880 - val_acc: 0.8009
Epoch 409/10000
6233/6233 [==============================] - 0s 52us/sample - loss: 0.3855 - acc: 0.8250 - val_loss: 0.3881 - val_acc: 0.7994
Epoch 410/10000
6233/6233 [==============================] - 0s 50us/sample - loss: 0.3853 - acc: 0.8307 - val_loss: 0.3877 - val_acc: 0.8009
Epoch 411/10000
6233/6233 [==============================] - 0s 50us/sample - loss: 0.3901 - acc: 0.8250 - val_loss: 0.3876 - val_acc: 0.7994
Epoch 412/10000
6233/6233 [==============================] - 0s 53us/sample - loss: 0.3885 - acc: 0.8256 - val_loss: 0.3879 - val_acc: 0.7994
Epoch 

Epoch 464/10000
6233/6233 [==============================] - 0s 53us/sample - loss: 0.3806 - acc: 0.8311 - val_loss: 0.3859 - val_acc: 0.7994
Epoch 465/10000
6233/6233 [==============================] - 0s 60us/sample - loss: 0.3814 - acc: 0.8325 - val_loss: 0.3854 - val_acc: 0.7994
Epoch 466/10000
6233/6233 [==============================] - 0s 53us/sample - loss: 0.3813 - acc: 0.8301 - val_loss: 0.3863 - val_acc: 0.7994
Epoch 467/10000
6233/6233 [==============================] - 0s 51us/sample - loss: 0.3782 - acc: 0.8299 - val_loss: 0.3857 - val_acc: 0.8009
Epoch 468/10000
6233/6233 [==============================] - 0s 50us/sample - loss: 0.3832 - acc: 0.8295 - val_loss: 0.3858 - val_acc: 0.8023
Epoch 469/10000
6233/6233 [==============================] - 0s 50us/sample - loss: 0.3805 - acc: 0.8296 - val_loss: 0.3855 - val_acc: 0.8009
Epoch 470/10000
6233/6233 [==============================] - 0s 51us/sample - loss: 0.3832 - acc: 0.8283 - val_loss: 0.3856 - val_acc: 0.8038
Epoch 

Epoch 522/10000
6233/6233 [==============================] - 0s 52us/sample - loss: 0.3760 - acc: 0.8288 - val_loss: 0.3842 - val_acc: 0.8038
Epoch 523/10000
6233/6233 [==============================] - 0s 52us/sample - loss: 0.3786 - acc: 0.8306 - val_loss: 0.3837 - val_acc: 0.8038
Epoch 524/10000
6233/6233 [==============================] - 0s 50us/sample - loss: 0.3799 - acc: 0.8341 - val_loss: 0.3837 - val_acc: 0.8052
Epoch 525/10000
6233/6233 [==============================] - 0s 50us/sample - loss: 0.3768 - acc: 0.8344 - val_loss: 0.3836 - val_acc: 0.8038
Epoch 526/10000
6233/6233 [==============================] - 0s 51us/sample - loss: 0.3759 - acc: 0.8344 - val_loss: 0.3838 - val_acc: 0.8038
Epoch 527/10000
6233/6233 [==============================] - 0s 51us/sample - loss: 0.3760 - acc: 0.8357 - val_loss: 0.3840 - val_acc: 0.8052
Epoch 528/10000
6233/6233 [==============================] - 0s 50us/sample - loss: 0.3793 - acc: 0.8307 - val_loss: 0.3841 - val_acc: 0.8052
Epoch 

Epoch 580/10000
6233/6233 [==============================] - 0s 56us/sample - loss: 0.3732 - acc: 0.8376 - val_loss: 0.3828 - val_acc: 0.8052
Epoch 581/10000
6233/6233 [==============================] - 0s 54us/sample - loss: 0.3726 - acc: 0.8376 - val_loss: 0.3822 - val_acc: 0.8081
Epoch 582/10000
6233/6233 [==============================] - 0s 57us/sample - loss: 0.3758 - acc: 0.8365 - val_loss: 0.3830 - val_acc: 0.8081
Epoch 583/10000
6233/6233 [==============================] - 0s 56us/sample - loss: 0.3742 - acc: 0.8343 - val_loss: 0.3823 - val_acc: 0.8052
Epoch 584/10000
6233/6233 [==============================] - 0s 53us/sample - loss: 0.3746 - acc: 0.8343 - val_loss: 0.3824 - val_acc: 0.8066
Epoch 585/10000
6233/6233 [==============================] - 0s 50us/sample - loss: 0.3752 - acc: 0.8323 - val_loss: 0.3825 - val_acc: 0.8081
Epoch 586/10000
6233/6233 [==============================] - 0s 50us/sample - loss: 0.3716 - acc: 0.8348 - val_loss: 0.3829 - val_acc: 0.8095
Epoch 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])

plt.title('model accuracy')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

In [ ]:
score = model.evaluate(x=X_test, y=y_test, verbose=1)
#score = model.evaluate(x=[X1_test, X2_test], y=y_test, verbose=1)

print("Test Score:", score[0])
print("Test ACC:", score[1])

In [ ]:
learning_rate = 0.00005
training_epochs = 1500
batch_size = 32
num_models = 10

tf.compat.v1.disable_eager_execution()

class Model:

    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()

    def _build_net(self):
        with tf.compat.v1.variable_scope(self.name):
            
            self.training = tf.keras.backend.placeholder(dtype = tf.bool)

            self.X = tf.keras.backend.placeholder(dtype = tf.float32, shape=(None,len(X_train.columns)))            
            self.Y = tf.keras.backend.placeholder(dtype = tf.float32, shape=(None,2))

            dense1 = tf.compat.v1.layers.dense(inputs=self.X, units = 10, activation=tf.nn.relu)
            dropout1 = tf.compat.v1.layers.dropout(inputs=dense1, rate=0.2, training=self.training)
            
            self.logits = tf.keras.layers.Dense(units = 2, activation = tf.nn.softmax)(dropout1)

        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.Y))
        self.optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)

        correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def predict(self, x_test, training=False):
        return self.sess.run(self.logits,
                             feed_dict={self.X: x_test, self.training: training})

    def get_accuracy(self, x_test, y_test, training=False):
        return self.sess.run(self.accuracy,
                             feed_dict={self.X: x_test,
                                        self.Y: y_test, self.training: training})

    def train(self, x_data, y_data, training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict={
            self.X: x_data, self.Y: y_data, self.training: training})

    
sess = tf.compat.v1.Session()


models = []
for m in range(num_models):
    models.append(Model(sess, "model" + str(m)))

sess.run(tf.compat.v1.global_variables_initializer())

print('Learning Started!')


for epoch in range(training_epochs):
    avg_cost_list = np.zeros(len(models))
    total_batch = int(len(X_train)/ batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = X_train[i * batch_size:(i+1) * batch_size], y_train[i * batch_size:(i+1) * batch_size]
        for m_idx, m in enumerate(models):
            c, _ = m.train(batch_xs, batch_ys)
            avg_cost_list[m_idx] += c / total_batch
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', avg_cost_list)

print('Learning Finished!')


test_size = len(y_test)
predictions = np.zeros([test_size, 2])
for m_idx, m in enumerate(models):
    print(m_idx, 'Accuracy:', m.get_accuracy(
        X_test, y_test))
    p = m.predict(X_test) 
    predictions += p

ensemble_correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y_test, 1))
ensemble_accuracy = tf.reduce_mean(tf.cast(ensemble_correct_prediction,  tf.float32))
print('Ensemble accuracy:', sess.run(ensemble_accuracy))